In [1]:
import tensorflow as tf

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.metrics
import os

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Training on C1 and Testing on C1

In [4]:
NS_case1 = pd.read_excel("../../NS_EOS_Case1.xlsx")
X_full = NS_case1.iloc[:, 0:7].values
y_full = NS_case1.iloc[:, [7,9,11]].values

from sklearn.cross_validation import train_test_split
X, X_test, y, y_test = train_test_split(X_full, y_full, test_size = 1/6, random_state = 0)

#Standardizing the data
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)
X_test = sc_X.transform(X_test)
sc_y = StandardScaler()
y = sc_y.fit_transform(y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Define the 2 HL model definition

In [21]:
def HL_2_model(i,j,result_new):
    #create model
    model = Sequential()  #creating sequential model; Dense specifies fully connected layer
    no_of_hidden_layers = 2
    node_in_each_layer = [i,j]
    model.add(Dense(node_in_each_layer[0],input_dim=7,activation='relu',kernel_initializer='normal')) #first hidden layer
    model.add(Dense(node_in_each_layer[1],activation='relu',kernel_initializer='normal')) #second hidden layer
    model.add(Dense(3,kernel_initializer='normal'))
    #compile model
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_squared_error'])
    #Train model
    train = model.fit(X,y,epochs=50,batch_size=16)  #size of dataset between 32-512
    #check performance
    y_pred = model.predict(X_test)

    y_pred = sc_y.inverse_transform(y_pred)
    #RMS value
    rms = sklearn.metrics.mean_squared_error(y_test, y_pred)**0.5
    rms_mass = sklearn.metrics.mean_squared_error(y_test[:,0], y_pred[:,0])**0.5
    rms_rad = sklearn.metrics.mean_squared_error(y_test[:,1], y_pred[:,1])**0.5
    rms_lam = sklearn.metrics.mean_squared_error(y_test[:,2], y_pred[:,2])**0.5

    #R2 value
    r2 = sklearn.metrics.r2_score(y_test,y_pred)
    
    #Selection Criterion
    flag=0
    ext_dir = ''
    if rms<30 and r2>0.875:     
        flag = 1
        ext_dir = 'Good_models'
    else:
        flag=2
        ext_dir = 'Bad_models'
    result_new = result_new.append({'Model':'ANN(2HL-'+str(i)+','+str(j)+' Nodes)', 'RMS':rms,'RMS_NS_Mass':rms_mass,'RMS_NS_Rad':rms_rad,
                            'RMS_Lam':rms_lam, 'R2':r2, 'Flag':flag}, ignore_index=True)
    #create and open folder   
    dire = os.path.join('2HL',ext_dir,'HL_nodes-'+str(i)+'_'+str(j))
    if not os.path.exists(dire):
        os.mkdir(dire)
    plt.plot(train.history['loss'])
    plt.title("Training loss curve (50 epochs)")
    direc = os.path.join(dire,'Training loss curve (50 epochs)')
    plt.savefig(direc)
    plt.close()
    #Plotting
    titles = ['NS_Mass','NS_Rad','Lam']
    for i in range(3):
        rms = 'RMSE:'+str((np.sqrt(sklearn.metrics.mean_squared_error(y_test[:,i], y_pred[:,i]))))
        fig, ax = plt.subplots()
        ax.scatter(y_test[:,i], y_pred[:,i])
        ax.plot([y_test[:,i].min(), y_test[:,i].max()], [y_test[:,i].min(), y_test[:,i].max()], 'k--', lw=4)
        ax.set_xlabel('Actual')
        ax.set_ylabel('Predicted')
        plt.title(titles[i]+';ANN(2HL-'+str(i)+','+str(j)+' Nodes);[Case1+Case2 on Case3] \n'+rms)
        direc = os.path.join(dire,titles[i])
        plt.savefig(direc)
        plt.close()
    return result_new
    

In [11]:
result_new = pd.DataFrame(columns=['Model','RMS','RMS_NS_Mass','RMS_NS_Rad','RMS_Lam','R2','Flag'])
for i in range(1,21):
    for j in range(1,21):
        result_new = HL_2_model(i,j,result_new)
result_new

Train on 1555 samples
Epoch 1/50
1555/1555 [==============================] - 0s 202us/sample - loss: 1.0004 - mean_squared_error: 1.0004
Epoch 2/50
1555/1555 [==============================] - ETA: 0s - loss: 1.0067 - mean_squared_error: 1.00 - 0s 59us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 3/50
1555/1555 [==============================] - 0s 57us/sample - loss: 1.0003 - mean_squared_error: 1.0003
Epoch 4/50
1555/1555 [==============================] - 0s 56us/sample - loss: 1.0004 - mean_squared_error: 1.0004
Epoch 5/50
1555/1555 [==============================] - 0s 57us/sample - loss: 1.0001 - mean_squared_error: 1.0001
Epoch 6/50
1555/1555 [==============================] - 0s 56us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 7/50
1555/1555 [==============================] - 0s 59us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 8/50
1555/1555 [==============================] - 0s 58us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch

Epoch 16/50
1555/1555 [==============================] - 0s 59us/sample - loss: 0.5939 - mean_squared_error: 0.5939
Epoch 17/50
1555/1555 [==============================] - 0s 57us/sample - loss: 0.5810 - mean_squared_error: 0.5810
Epoch 18/50
1555/1555 [==============================] - ETA: 0s - loss: 0.5893 - mean_squared_error: 0.58 - 0s 57us/sample - loss: 0.5691 - mean_squared_error: 0.5691
Epoch 19/50
1555/1555 [==============================] - 0s 59us/sample - loss: 0.5587 - mean_squared_error: 0.5587
Epoch 20/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.5493 - mean_squared_error: 0.5493
Epoch 21/50
1555/1555 [==============================] - 0s 60us/sample - loss: 0.5412 - mean_squared_error: 0.5412
Epoch 22/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.5338 - mean_squared_error: 0.5338
Epoch 23/50
1555/1555 [==============================] - 0s 59us/sample - loss: 0.5274 - mean_squared_error: 0.5274
Epoch 24/50
1555/155

1555/1555 [==============================] - 0s 55us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 33/50
1555/1555 [==============================] - 0s 55us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 34/50
1555/1555 [==============================] - 0s 56us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 35/50
1555/1555 [==============================] - 0s 55us/sample - loss: 1.0003 - mean_squared_error: 1.0003
Epoch 36/50
1555/1555 [==============================] - 0s 56us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 37/50
1555/1555 [==============================] - 0s 55us/sample - loss: 1.0003 - mean_squared_error: 1.0003
Epoch 38/50
1555/1555 [==============================] - 0s 56us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 39/50
1555/1555 [==============================] - 0s 56us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 40/50
1555/1555 [==============================] - 0s 55us/sample - loss: 1.00

1555/1555 [==============================] - 0s 56us/sample - loss: 1.0001 - mean_squared_error: 1.0001
Epoch 48/50
1555/1555 [==============================] - 0s 56us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 49/50
1555/1555 [==============================] - 0s 56us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 50/50
1555/1555 [==============================] - 0s 56us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Train on 1555 samples
Epoch 1/50
1555/1555 [==============================] - 0s 255us/sample - loss: 0.9990 - mean_squared_error: 0.9990
Epoch 2/50
1555/1555 [==============================] - 0s 56us/sample - loss: 0.9646 - mean_squared_error: 0.9646
Epoch 3/50
1555/1555 [==============================] - 0s 55us/sample - loss: 0.8318 - mean_squared_error: 0.8318s - loss: 0.8492 - mean_squared_error: 0.84
Epoch 4/50
1555/1555 [==============================] - ETA: 0s - loss: 0.7028 - mean_squared_error: 0.70 - 0s 58us/sample - loss: 0.7141

1555/1555 [==============================] - 0s 56us/sample - loss: 0.5387 - mean_squared_error: 0.5387
Epoch 14/50
1555/1555 [==============================] - 0s 56us/sample - loss: 0.5312 - mean_squared_error: 0.5312
Epoch 15/50
1555/1555 [==============================] - 0s 55us/sample - loss: 0.5244 - mean_squared_error: 0.5244
Epoch 16/50
1555/1555 [==============================] - 0s 55us/sample - loss: 0.5186 - mean_squared_error: 0.5186
Epoch 17/50
1555/1555 [==============================] - 0s 56us/sample - loss: 0.5137 - mean_squared_error: 0.5137
Epoch 18/50
1555/1555 [==============================] - 0s 56us/sample - loss: 0.5089 - mean_squared_error: 0.5089
Epoch 19/50
1555/1555 [==============================] - 0s 55us/sample - loss: 0.5052 - mean_squared_error: 0.5052s - loss: 0.4822 - mean_squared_error: 0.48
Epoch 20/50
1555/1555 [==============================] - 0s 55us/sample - loss: 0.5015 - mean_squared_error: 0.5015
Epoch 21/50
1555/1555 [==================

...

1555/1555 [==============================] - ETA: 0s - loss: 0.0823 - mean_squared_error: 0.08 - 0s 58us/sample - loss: 0.1020 - mean_squared_error: 0.1020
Epoch 26/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.1003 - mean_squared_error: 0.1003
Epoch 27/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.0997 - mean_squared_error: 0.0997
Epoch 28/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.0982 - mean_squared_error: 0.0982
Epoch 29/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.0974 - mean_squared_error: 0.0974
Epoch 30/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.0972 - mean_squared_error: 0.0972
Epoch 31/50
1555/1555 [==============================] - ETA: 0s - loss: 0.0878 - mean_squared_error: 0.08 - 0s 58us/sample - loss: 0.0953 - mean_squared_error: 0.0953
Epoch 32/50
1555/1555 [==============================] - 0s 57us/sample - loss: 0.0953 - mean_sq

1555/1555 [==============================] - 0s 58us/sample - loss: 0.0863 - mean_squared_error: 0.0863
Epoch 42/50
1555/1555 [==============================] - 0s 59us/sample - loss: 0.0853 - mean_squared_error: 0.0853
Epoch 43/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.0840 - mean_squared_error: 0.0840
Epoch 44/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.0841 - mean_squared_error: 0.0841
Epoch 45/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.0830 - mean_squared_error: 0.0830s - loss: 0.1053 - mean_squared_error: 0.10
Epoch 46/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.0822 - mean_squared_error: 0.0822
Epoch 47/50
1555/1555 [==============================] - ETA: 0s - loss: 0.0812 - mean_squared_error: 0.08 - 0s 63us/sample - loss: 0.0823 - mean_squared_error: 0.0823
Epoch 48/50
1555/1555 [==============================] - 0s 57us/sample - loss: 0.0814 - mean_squared_err

1555/1555 [==============================] - 0s 59us/sample - loss: 0.1501 - mean_squared_error: 0.1501
Epoch 9/50
1555/1555 [==============================] - 0s 59us/sample - loss: 0.1411 - mean_squared_error: 0.1411
Epoch 10/50
1555/1555 [==============================] - 0s 59us/sample - loss: 0.1340 - mean_squared_error: 0.1340
Epoch 11/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.1282 - mean_squared_error: 0.1282
Epoch 12/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.1227 - mean_squared_error: 0.1227
Epoch 13/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.1195 - mean_squared_error: 0.1195
Epoch 14/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.1168 - mean_squared_error: 0.1168
Epoch 15/50
1555/1555 [==============================] - 0s 60us/sample - loss: 0.1153 - mean_squared_error: 0.1153
Epoch 16/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.111

Epoch 26/50
1555/1555 [==============================] - 0s 59us/sample - loss: 0.1028 - mean_squared_error: 0.1028
Epoch 27/50
1555/1555 [==============================] - 0s 59us/sample - loss: 0.1030 - mean_squared_error: 0.1030
Epoch 28/50
1555/1555 [==============================] - ETA: 0s - loss: 0.0900 - mean_squared_error: 0.09 - 0s 60us/sample - loss: 0.1006 - mean_squared_error: 0.1006
Epoch 29/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.1008 - mean_squared_error: 0.1008
Epoch 30/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.0993 - mean_squared_error: 0.0993
Epoch 31/50
1555/1555 [==============================] - 0s 58us/sample - loss: 0.0988 - mean_squared_error: 0.0988
Epoch 32/50
1555/1555 [==============================] - 0s 59us/sample - loss: 0.0971 - mean_squared_error: 0.0971
Epoch 33/50
1555/1555 [==============================] - ETA: 0s - loss: 0.1238 - mean_squared_error: 0.12 - 0s 59us/sample - loss: 0.09

,Model,RMS,RMS_NS_Mass,RMS_NS_Rad,RMS_Lam,R2,Flag
0,"ANN(2HL-1,1 Nodes)",68.760005,0.112645,0.868756,119.092601,-0.000730,2
1,"ANN(2HL-1,2 Nodes)",35.625294,0.105115,0.534464,61.702415,0.493692,2
2,"ANN(2HL-1,3 Nodes)",68.761885,0.112649,0.868751,119.095857,-0.000764,2
3,"ANN(2HL-1,4 Nodes)",68.768843,0.112637,0.868781,119.107909,-0.000788,2
4,"ANN(2HL-1,5 Nodes)",34.644535,0.103133,0.565262,60.003343,0.494446,2
5,"ANN(2HL-1,6 Nodes)",34.804392,0.104130,0.553240,60.280346,0.494169,2
6,"ANN(2HL-1,7 Nodes)",32.219018,0.106862,0.524436,55.802410,0.504955,2
7,"ANN(2HL-1,8 Nodes)",34.592202,0.103249,0.561504,59.912730,0.495944,2
8,"ANN(2HL-1,9 Nodes)",34.542817,0.103178,0.559918,59.827205,0.497360,2
9,"ANN(2HL-1,10 Nodes)",32.266665,0.106858,0.524279,55.884942,0.504832,2


In [20]:
#Segregating bad and good models
result_new_good1 = result_new[result_new['R2']>0.875 ]  
result_new_good1 = result_new_good1[result_new_good1['RMS']<30]
result_new_good = result_new_good1.reset_index()
result_new_good = result_new_good.drop(['index','Flag'],axis=1)
result_new_good

,Model,RMS,RMS_NS_Mass,RMS_NS_Rad,RMS_Lam,R2
0,"ANN(2HL-14,18 Nodes)",27.306187,0.026483,0.348853,47.294410,0.875179
1,"ANN(2HL-14,19 Nodes)",27.064347,0.023133,0.337561,46.875603,0.883902
2,"ANN(2HL-16,17 Nodes)",27.731873,0.025797,0.335864,48.031831,0.878398
3,"ANN(2HL-16,18 Nodes)",27.598346,0.027890,0.338865,47.800528,0.875069
4,"ANN(2HL-16,19 Nodes)",27.431938,0.022613,0.332540,47.512341,0.884600
5,"ANN(2HL-17,15 Nodes)",28.587918,0.023932,0.337952,49.514567,0.876813
6,"ANN(2HL-17,16 Nodes)",28.206804,0.023511,0.338937,48.854436,0.878571
7,"ANN(2HL-17,17 Nodes)",27.759959,0.027265,0.335676,48.080481,0.876295
8,"ANN(2HL-17,18 Nodes)",28.894337,0.022936,0.343368,50.045277,0.875167
9,"ANN(2HL-17,19 Nodes)",28.320515,0.023766,0.342476,49.051370,0.876735


In [22]:
#Writing in file
direc1 = os.path.join('2HL','All_Model_details.xlsx')
result_new_bad.to_excel(direc1)
direc2 = os.path.join('2HL','Good_Model_details.xlsx')
result_new_good.to_excel(direc2)


In [34]:
list_of_Good_models = []
names = result_new_good['Model'].tolist()
for x in names:
    ij = x[8:-7].split(',')
    list_of_Good_models.append((int(ij[0]),int(ij[1])))
list_of_Good_models

[(14, 18),
 (14, 19),
 (16, 17),
 (16, 18),
 (16, 19),
 (17, 15),
 (17, 16),
 (17, 17),
 (17, 18),
 (17, 19),
 (18, 13),
 (18, 14),
 (18, 16),
 (18, 17),
 (18, 19),
 (18, 20),
 (19, 12),
 (19, 16),
 (19, 18),
 (19, 19),
 (19, 20),
 (20, 13),
 (20, 15),
 (20, 16),
 (20, 18),
 (20, 19),
 (20, 20)]

In [35]:
list_of_Good_models= [(14, 18),(14, 19),(16, 17),(16, 18),(16, 19),(17, 15),(17, 16),(17, 17),(17, 18),(17, 19),(18, 13),
                      (18, 14),(18, 16),(18, 17),(18, 19),(18, 20),(19, 12),(19, 16),(19, 18),(19, 19),(19, 20),(20, 13),
                      (20, 15),(20, 16),(20, 18),(20, 19),(20, 20)]